#### Working with data

In [10]:
import pandas as pd
from pathlib import Path

In [11]:
input_folder = Path("../data/raw_data")
output_folder = Path("../data/processed_data")
output_folder.mkdir(exist_ok=True)

In [12]:
replacements = {
    " (1)": "",
    " (2)": "",
    " (3)": "",
}

In [13]:
def replace_invalid_chars(text):
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

In [14]:
def determine_region_type(code):
    code = str(code).zfill(7)
    if code == "0000000":  # Country
        return "kraj"
    elif code[0:1] != "00" and code[2:] == "00000":  # Voivodeship
        return "województwo"
    elif code[2:3] != "00" and code[4:] == "000":  # District
        return "powiat"
    elif code.isdigit() and len(code) == 7 and code[4:] != "000":  # City
        return "miasto"
    else:
        return "nieznane"

In [15]:
def analyze_and_clean_file(file_path, columns_to_analyze):
    data = pd.read_csv(file_path, delimiter=";")
    
    data["Nazwa"] = data["Nazwa"].apply(lambda x: replace_invalid_chars(x) if isinstance(x, str) else x)
    
    data = data.drop(columns=["Atrybut"], errors="ignore")  
    data["Jednostka miary"] = data["Jednostka miary"].str.strip() 
    data["Region_Type"] = data["Kod"].apply(determine_region_type) 
    
    analysis_results = {}
    for column in columns_to_analyze:
        analysis_results[column] = data[column].unique()
        print(f"Distinct values in '{column}':")
        print(analysis_results[column])
        print("\n")

    print("Region Type Counts:")
    print(data["Region_Type"].value_counts())
    
    cleaned_file_path = output_folder / f"correct_{file_path.name}"
    data.to_csv(cleaned_file_path, index=False, sep=";")
    print(f"Cleaned file saved to {cleaned_file_path}")
    return data, analysis_results

In [16]:
# Fields to analyse
files_and_columns = {
    "raw_road_accidents_and_their_victims_1999_2023.csv": ["Wypadki drogowe", "Rok"],
    "raw_road_accidents_by_major_causes_2014_2023.csv": ["Ważniejsze przyczyny wypadków", "Rok"],
    "raw_road_accidents_intoxicated_road_users_2014_2023.csv": ["Nietrzeźwi sprawcy", "Rok"],
    "raw_road_accidents_wehicle_type_2014_2023.csv": ["Wypadki drogowe według pojazdu sprawcy", "Rok"]
}

In [17]:
for file_name, columns in files_and_columns.items():
    print(f"Analyzing file: {file_name}")
    file_path = input_folder / file_name
    analyze_and_clean_file(file_path, columns)

Analyzing file: raw_road_accidents_and_their_victims_1999_2023.csv
Distinct values in 'Wypadki drogowe':
['wypadki ogółem' 'ofiary wypadków ogółem' 'ofiary śmiertelne' 'ranni']


Distinct values in 'Rok':
[1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


Region Type Counts:
miasto         227400
powiat          38000
województwo      1600
kraj              100
Name: Region_Type, dtype: int64
Cleaned file saved to ..\data\processed_data\correct_raw_road_accidents_and_their_victims_1999_2023.csv
Analyzing file: raw_road_accidents_by_major_causes_2014_2023.csv
Distinct values in 'Ważniejsze przyczyny wypadków':
['ogółem' 'wina kierujących pojazdami - ogółem'
 'wina kierujących pojazdami - niedostosowanie prędkości do warunków ruchu'
 'wina kierujących pojazdami - nieprzestrzeganie pierwszeństwa przejazdu'
 'wina kierujących pojazdami - nieprawidłowe wyprzedzanie'
 'wina kierujących pojazdami - nieprawidłowe za